# Lab Fees

In [ ]:
#read in the data and the file with all the duplicates
import pandas as pd
import numpy as np

lab_dups = pd.read_csv('lab_dups.csv',error_bad_lines=False)
lab_newcols = lab_dups['newcols']
del lab_dups['newcols']

lab = pd.read_csv('lab.csv', encoding = "ISO-8859-1", sep = ";")
lab = lab.replace("N/A", np.nan)

In [23]:
def comb_dups(dups, dat, newcols)
    for i in range(0, dups.shape[0]-1): #for each row in
        newcol_items = [] #come up with a list that will hold valid value among duplicates
    
        colslist = dups.iloc[i].unique() #obtain the valid dat column names
        colslist = [x for x in colslist if str(x) != 'nan'] 
        for row in range(0,dat.shape[0]-1): #for each row in dat
            vals = dat[colslist].iloc[row].unique() 
                #create a list of values concerning duplicate columns
            vals = [x for x in vals if str(x) != 'nan'] #strip it of NaNs
            if len(vals) == 0: #if all of them happened to have been NaNs
                newcol_items.append(np.nan) #record the final value as a NaN
            else: 
                vals = [float(v) for v in vals] #else: take the max value among them
                newcol_items.append(np.nanmax(vals))

        dat = dat.drop(dat.ix[:,colslist].head(0).columns, axis=1) #drop all the duplicate columns
        dat[newcols[i]] = newcol_items #add list of valid values to list for output dataframe
        return(dat)
    
lab = comb_dups(lab_dups, lab, lab_newcols)

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  from ipykernel import kernelapp as app


In [24]:
lab.to_csv('lab_cleaned.csv',index = False)

### Data Analysis

In [2]:
def summarize(dat):
    meds = []
    mean = []
    sd = []
    median = []
    mini = []
    maxi = []
    rangeo = []
    n = []
    high_outliers = []
    low_outliers = []

    for col in dat.columns.values[dat.columns.values != 'hosp_name']:
    
        vals = [i for i in dat[] if str(i) != 'nan']
    
        if len(vals) >= 10:
            meds.append(col)
            mean.append(np.mean(vals))
            sd.append(np.std(vals))
            median.append(np.median(vals))
            mini.append(np.nanmin(vals))
            maxi.append(np.nanmax(vals))
            rangeo.append(np.nanmax(vals) - np.nanmin(vals))
            n.append(len(vals))
    
            high_outlier = []
            low_outlier = []
    
            for hosp in dat['hosp_name']:
                if str(dat[col].loc[dat.hosp_name == hosp].values[0]) != 'nan':
                    val = dat[col].loc[dat.hosp_name == hosp].values[0]

                    if np.less(np.mean(vals) + 1.96*np.std(vals), val):
                        outlier = [hosp, ": ",str(val)]
                        high_outlier.append("".join(outlier))
            
                    if np.less(val, np.mean(vals) - 1.96*np.std(vals)):
                        outlier = [hosp, ": ",str(val)]
                        low_outlier.append("".join(outlier))
    
            high_outliers.append(high_outlier)
            low_outliers.append(low_outlier)

    summary = pd.DataFrame({'meds': meds,
                       'mean': mean,
                       'sd': sd,
                       'median': median,
                       'min': mini,
                       'max': maxi,
                       'range': rangeo,
                       'N': n,
                       'high outliers': high_outliers,
                       'low outliers': low_outliers})
    return(summary)

summary1 = summarize(lab)

In [140]:
summary1.to_csv('summary1.csv')

### Creating a summary of the outlying values

In [11]:
lab_code = pd.read_csv('lab_code.csv')

In [12]:
lab_code.head()

,Variable,Label Name
0,acid_phos,Acid Phosphate
1,alka_phos,Alkaline Phosphatase (ALP)
2,anti.hbs,Anti-HBs
3,anti.hbe,Anti-HBe
4,anti.hbc,Anti-HBc


In [40]:
def summarize_outliers(summary, dat, code):
    hosps = dat['hosp_name']

    outlier_lists = []

    for hosp in hosps:
        outs = []
        for row in range(0, summary.shape[0]- 1):
            outs = '\t'.join(summary['high outliers'].iloc[row])
            if hosp in outs:
                var = summary['meds'].iloc[row]
                outs.append(np.array(code['Label Name'].loc[code.Variable == var])[0])
        outlier_lists.append(outs)
    
    summary_outs = pd.DataFrame({'hospitals': hosps,
                             'meds to recheck': outlier_lists})
    return(summary_outs)
    
summary1_outs = summarize_outliers(summary1, lab, lab_code)

In [42]:
summary1_outs.to_csv('summary1_outs.csv')

# Diagnosis Fees